In [1]:
import pandas as pd
import sys
from pathlib import Path
import urllib.request
import os.path
import json

In [2]:
Path("data").mkdir(parents=True, exist_ok=True)

In [5]:
if (not os.path.exists("data/content.json")):
    urllib.request.urlretrieve("https://ificloud.xsitepool.tu-freiberg.de/index.php/s/LME56sPXBCwxnLP/download/content.json", "data/content.json")

In [6]:
data_folder = "data"
json_file_name = "content.json"
input_file = Path.cwd() / data_folder / json_file_name

with open(input_file, 'r', encoding='utf-8') as json_data:
    raw_data = json.load(json_data)

In [7]:
ressource_types = {"files": "file", 
                   "learning_resources": "lr"}

oer_data_set = list()
for ressource_type in ressource_types:
    data = raw_data[ressource_type]
    print(f"{ressource_type} - {len(data)} entries")
    for entry in data:
        entry['type'] = ressource_types[ressource_type]
        oer_data_set.append(entry)

files - 11322 entries
learning_resources - 3828 entries


In [8]:
df = pd.DataFrame(oer_data_set)

In [9]:
def getLastValue(aList):
    if  isinstance(aList, list):
        return aList[-1]
    else:
        return "unknown"

# Extract file extension
df['file_type'] = df.filename.str.split('.').apply(getLastValue)

# Normalize different writing formates for file extensions
df['file_type'] = df['file_type'].str.lower()

In [10]:
daten = []
i=0
for index, row in df[df.type == "file"].iterrows():
    
    if row.file_type == "pdf":        
        daten.append(row.oer_permalink)

In [11]:
series = pd.Series(daten)
series.to_csv("data/PDFURLs.csv", header=False, index=False)